# Data Preprocessing for ML


In [ ]:
# Dividing numeric and categorical variables
num_vars = [var for var in df_cleaned.columns if df_cleaned[var].dtype in ['int64', 'float64']]
cat_vars = [var for var in df_cleaned.columns if df_cleaned[var].dtype not in ['int64', 'float64']]

### Outlier Treatment


In [ ]:
def find_numeric_outliers(df_cleaned):
    outliers = {}
    for col in df_cleaned:
      if df_cleaned[col].dtype in ['float64', 'int64']:
        quartile_1, quartile_3 = df_cleaned[col].quantile([.25, .75])
        interquartile_range = quartile_3 - quartile_1
        lower_bound = quartile_1 - (1.5 * interquartile_range)
        upper_bound = quartile_3 + (1.5 * interquartile_range)
        outlier_rows = df_cleaned[(df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)]
        outliers[col] = {
          'lower_bound': lower_bound,
          'upper_bound': upper_bound,
          'outliers': outlier_rows[col].tolist()
        }
    return outliers

def create_outlier_matrix(df_cleaned):
    outlier_mat = np.zeros_like(df_cleaned)
    for j ,var in enumerate(df_cleaned.columns):
        if var in num_vars:
            column = df_cleaned[var]
            q1 = column.quantile(0.25)
            q3 = column.quantile(0.75)
            iqr = q3 - q1
            l1 = q1 - 1.5*iqr
            l2 = q3 + 1.5*iqr
            outlier_index = column[(column<l1) | (column>l2)].index
            for i in outlier_index:
                outlier_mat[i,j] = 1
    return outlier_mat

sns.heatmap(outlier_mat)

# Removing time-based variables, some highly skewed features, and variables with clinically expected distributions.
num_vars = [var for var in num_vars if var not in all_to_remove]

# The Fisher method for two-tailed test
def cocor(x1,y1, x2,y2):
    xy1 = x1.corr(y1, method='spearman')
    xy2 = x2.corr(y2, method='spearman')
    n1 = len(x1)
    n2 = len(x2)
    xy_z = 0.5 * np.log((1 + xy1)/(1 - xy1))
    ab_z = 0.5 * np.log((1 + xy2)/(1 - xy2))
    if n2 is None:
        n2 = n1
    se_diff_r = np.sqrt(1/(n1 - 3) + 1/(n2 - 3))
    diff = xy_z - ab_z
    z = abs(diff / se_diff_r)
    p = (1 - norm.cdf(z)) * 2
    return z, p

# Test correlations
p_values_corr = []
for j, col in enumerate(df_cleaned.columns):
    if col in num_vars:
        s = df_cleaned[col]
        s_withot_outliers = s.filter(items = [i for i in range(df_cleaned.shape[0]) if outlier_mat[i,j] == 0])
        _,p = cocor(s , df_cleaned.any_vte_result , s_withot_outliers , df_cleaned.any_vte_result)
        print(col, '----', 'corr with outliers: ', s.corr(df_cleaned.any_vte_result), 'corr without outliers: ', s_withot_outliers.corr(df_cleaned.any_vte_result))
        p_values_corr.append(p)

# Testing distributions with Kolmogorov-Smirnov test (stats.kstest())
p_values_dist = []
for j, col in enumerate(df_cleaned.columns):
    if col in num_vars:
        s = df_cleaned[col]
        s_without_outliers = s[outlier_mat[:, j] == 0]
        _, p = stats.kstest(s, s_without_outliers)
        p_values_dist.append(p)

# Creating a decision table for outlier treatment (num_vars)
decision_table['p-value dist'] = p_values_dist
decision_table['distribution changed'] = ['yes' if p < 0.05 else 'no' for p in p_values_dist]
def drop(df_cleaned):
    if ((df_cleaned['correlation changed'] == 'yes') & (df_cleaned['distribution changed'] == 'yes')):
        return 'no'
    else:
        return 'yes'
decision_table['drop'] = decision_table.apply(drop, axis = 1)

# Converting the "yes" features to np.nan
vars_with_outliers_to_drop = decision_table[decision_table['drop'] == 'yes'].index
for j, col in enumerate(df_cleaned.columns):
    if col in vars_with_outliers_to_drop:
        outlier_rows = [i for i in range(df_cleaned.shape[0]) if outlier_mat[i,j] == 1 ]
        df_cleaned.iloc[outlier_rows,j] = np.nan

# For highly skewed features features
def replace_outliers_with_nan(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return series.where((series >= lower_bound) & (series <= upper_bound), np.nan)

for col in cols_to_clean:
df_cleaned[col] = replace_outliers_with_nan(df_cleaned[col])

In [ ]:
# A new Copy of the dataset
df_vte = df_cleaned.copy()

In [ ]:
# Creating the target outcome for the regression task
df_vte["days_to_any_vte_result"] = df_cleaned[["days_to_vte", "days_to_pe", "days_to_dvt"]].min(axis=1, skipna=True)
df_vte["days_to_end_of_study"] = 540

df_vte["days_to_any_vte_result"] = df_vte[[
    "days_to_any_vte_result",
    "days_to_end_of_study"
]].min(axis=1)

In [ ]:
# Categorical and numerical groups
num_vars = [var for var in df_vte.columns if df_vte[var].dtype.name != 'category']
cat_vars = [var for var in df_vte.columns if df_vte[var].dtype.name == 'category']

### Missing Value Handling


In [ ]:
# Calculating the percentage
missing_percentages = df_vte.isnull().sum() * 100 / len(df_vte)
missing_percentages = missing_percentages[missing_percentages > 0]

# A total of 39 variables were found to have missing values, all with more than 60% missingness
# Of these, 34 were numeric lab results and 5 were categorical variables related to rehabilitation admissions
# Due to manual handaling of these variables, only a few examples will be presented

# Example 1
def categorize_pulse(value):
    if pd.isna(value):
        return 0
    elif value <= 59 or value > 100:
        return 1  # Bradycardia/ Tachycardia
    else:
        return 2  # normal

# Example 2
def categorize_saturation(value):
    if pd.isna(value):
        return 0 # missing
    else:
        return 1  # not missing

# Example 3
def categorize_dbp(value):
    if pd.isna(value):
        return 0
    elif value < 60:
        return 1  # Low DBP
    elif value <= 80:
        return 2  # Normal DBP
    else:
        return 3  # High DBP

# Example 4
def categorize_hb(value):
    if pd.isna(value):
        return 0
    elif value < 12:
        return 1  # Low (possible anemia)
    elif value <= 16:
        return 2  # Normal
    else:
        return 3  # High

# Example 5
def categorize_hct(value):
    if pd.isna(value):
        return 0
    elif value < 36 or value > 50:
        return 1  # Abnormal - Low (possible anemia)/ High
    else:
        return 2  # Normal

### Encoding

**Categorical variables were encoded using a combination of binning, one-hot encoding, and ordinal encoding, depending on their type and distribution.**

In [ ]:
# Binning Example:

# Charlson score
df_vte['charlson_comorbidity_index-charlson score total'] = df_vte['charlson_comorbidity_index-charlson score total'].apply(
    lambda x: 2 if x >= 2 else x)

df_vte['charlson_comorbidity_index-charlson score total'] = df_vte['charlson_comorbidity_index-charlson score total'].astype('category')

# One-Hot Encoding Example:

# Gender
df_vte['gender_m'] = [1 if x == 'Male' else 0 for x in df_vte.gender]
df_vte['gender_f'] = [1 if x == 'Female' else 0 for x in df_vte.gender]
df_vte['gender_m'] = df_vte['gender_m'].astype('category')
df_vte['gender_f'] = df_vte['gender_f'].astype('category')
df_vte.drop('gender', axis=1, inplace=True)

# Ordinal Encoding Example:

# Socioeconomic score
socioeconomic_order = [['no data', 'Very Low', 'Low', 'Medium', 'High', 'Very High']]
encoder_socio = OrdinalEncoder(categories=socioeconomic_order)
df_vte['socioeconomic score five level scale'] = encoder_socio.fit_transform(
    df_vte[['socioeconomic score five level scale']]
).astype(int)

In [ ]:
# Regression model dataset
df_vte_regression = df_vte.copy()

### Feature Selection

In [ ]:
# Classification:

p_values = []
columns = list(df_vte.columns)
columns.remove('any_vte_result')
for var in columns:
    if var in num_vars:
        # For numeric variables, perform Mann-Whitney U test
        x1 = df_vte.query('any_vte_result == 1')[var]
        x2 = df_vte.query('any_vte_result == 0')[var]
        _, p = stats.mannwhitneyu(x1, x2)
    else:
        # For categorical variables, perform Chi-square test
        _, p, _, _ = stats.chi2_contingency(pd.crosstab(df_vte[var], df_vte.any_vte_result))

    p_values.append(p)
selection = pd.DataFrame(p_values, index=columns, columns=['p_value'])
selection['keep'] = ['yes' if p <= 0.05 else 'no' for p in selection.p_value]

cols_to_drop = selection.query("keep == 'no'").index
df_vte.drop(cols_to_drop, axis=1, inplace=True)

# Regression:
target = 'days_to_any_vte_result'
num_vars_reg = [var for var in df_vte_regression.columns
                if df_vte_regression[var].dtype.name != 'category' and var != target]

cat_vars_reg = [var for var in df_vte_regression.columns
                if df_vte_regression[var].dtype.name == 'category']
p_values = []
for var in num_vars_reg + cat_vars_reg:
    if var == target:
        continue
    if var in num_vars_reg:
        # Spearman correlation for numeric vs. continuous
        coef, p = stats.spearmanr(df_vte_regression[var], df_vte_regression[target], nan_policy='omit')
    else:
        # Kruskal–Wallis test for categorical vs. continuous
        groups = [df_vte_regression[df_vte_regression[var] == level][target].dropna()
                  for level in df_vte_regression[var].unique()]
        _, p = stats.kruskal(*groups)

    p_values.append((var, p))
selection_reg = pd.DataFrame(p_values, columns=['variable', 'p_value'])
selection_reg['keep'] = selection_reg['p_value'].apply(lambda x: 'yes' if x <= 0.05 else 'no')
selection_reg.set_index('variable', inplace=True)
cols_to_drop_reg = selection_reg.query("keep == 'no'").index
df_vte_regression.drop(cols_to_drop_reg, axis=1, inplace=True)

## Classification

### Data Division

In [ ]:
# For train/dev/test
x = df_vte.drop(['any_vte_result'], axis=1)
y = df_vte['any_vte_result']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 14)
x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2, random_state = 14)

# For SKfold
X = df_vte.drop(columns=['any_vte_result'])
y = df_vte['any_vte_result']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=14)

### Scaling

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_dev = scaler.transform(x_dev)
x_test = scaler.transform(x_test)

### Data Imbalance Methods

In [ ]:
# Resampling for train/dev/test method
over = RandomOverSampler(sampling_strategy='all', random_state=14)
under = RandomUnderSampler(sampling_strategy='majority', random_state=14)
smote = SMOTE(random_state=14)
smote_enn = SMOTEENN(random_state=14)
train_over_x , train_over_y = over.fit_resample(x_train, y_train)
train_under_x , train_under_y = under.fit_resample(x_train, y_train)
train_smote_x , train_smote_y  = smote.fit_resample(x_train, y_train)
train_smote_enn_x, train_smote_enn_y = smote_enn.fit_resample(x_train, y_train)

# Resampling for SKfold
sampling_methods = {
    'Original': (X_train, y_train),
    'Random Over': over.fit_resample(X_train, y_train),
    'Random Under': under.fit_resample(X_train, y_train)
#     'SMOTE': smote.fit_resample(X_train, y_train),
#     'SMOTEENN': smote_enn.fit_resample(X_train, y_train)
}

for method_name, (X_samp, y_samp) in sampling_methods.items():
    print(f"\n=== {method_name} ===")
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    train_auc_scores = []
    val_auc_scores = []

    for train_idx, val_idx in skf.split(X_samp, y_samp):
        X_t, X_val = X_samp.iloc[train_idx], X_samp.iloc[val_idx]
        y_t, y_val = y_samp.iloc[train_idx], y_samp.iloc[val_idx]

        model = LogisticRegression(penalty='l2', max_iter=10000, random_state=0)
        model
        model.fit(X_t, y_t)
        y_t_prob = model.predict_proba(X_t)[:, 1]
        y_val_prob = model.predict_proba(X_val)[:, 1]

        train_auc = roc_auc_score(y_t, y_t_prob)
        val_auc = roc_auc_score(y_val, y_val_prob)

        train_auc_scores.append(train_auc)
        val_auc_scores.append(val_auc)

### Metrics for Modeling

In [ ]:
def classificationMetrics(y, y_proba):
    if y_proba.ndim == 2:
        y_proba = y_proba[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)
    auc = metrics.roc_auc_score(y, y_proba)
    f1 = metrics.f1_score(y, y_pred)
    recall = metrics.recall_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred)

    return {
        'AUC': auc,
        'F1': f1,
        'Recall': recall,
        'Precision': precision
    }

## Regression

In [ ]:
# From 540 days to 77 weeks
df_vte_regression['days_to_any_vte_result'] = (
    df_vte_regression['days_to_any_vte_result'].astype(int) // 7)

### Data Division

In [ ]:
X_reg = df_vte_regression.drop(columns='days_to_any_vte_result')
Y_reg = df_vte_regression['days_to_any_vte_result']
X_train_dev_reg, X_test_reg, y_train_dev_reg, y_test_reg = train_test_split(X_reg, Y_reg, test_size=0.2, random_state=14)
X_train_reg, X_dev_reg, y_train_reg, y_dev_reg = train_test_split(X_train_dev_reg, y_train_dev_reg, test_size=0.2, random_state=42)

# Lower weights for week 77 (censored)
weights_train = np.where(y_train_reg == 77, 0.1, 1.0)

### Metrics for modeling

In [ ]:
def regressionMetrics(y, yhat):
    yhat_clipped = np.maximum(yhat, 0)
    y_clipped = np.maximum(y, 0)

    return {
        'MAE': metrics.mean_absolute_error(y, yhat),
        'RMSE': metrics.mean_squared_error(y, yhat, squared=False),
        'MSLE': metrics.mean_squared_log_error(y_clipped, yhat_clipped),
    }